In [1]:
import pandas as pd
import numpy as np
import math
import statistics

raw_data = pd.read_csv("CompleteDataSet_training_competition_1500.csv", sep=",")

data = raw_data[['WristAccelerometer x-axis', 'WristAccelerometer y-axis', 'WristAccelerometer z-axis', 'Activity']]

n_step = 1500
n_batch = int(data.shape[0] / n_step)

# variables used in the paper
k = 15
t = 3
K = 300

xyz = 3

data = data.as_matrix()
calculate = np.zeros([n_batch * int(k / t), 15 + 1])

# for each data
for i in range(n_batch):
    
    # for each sliding window
    for j in range(int(k / t)):
        calculate[int(k / t) * i + j][15] = data[i * n_step][xyz]
        
        avm = np.zeros([K])
        aamvsum = 0
        xyzsum = np.zeros([K])
        mpi = 0
        mvi = 0
        saa = np.zeros([xyz])
        sma = 0
        ta = np.zeros([K, xyz])
        tasum = np.zeros([xyz])
        tasquare = np.zeros([xyz])
        for m in range(K):
            avm[m] = math.sqrt(pow(data[i * n_step + j * K + m][0], 2) + pow(data[i * n_step + j * K + m][1], 2) + pow(data[i * n_step + j * K + m][2], 2))
            xyzsum[m] = data[i * n_step + j * K + m][0] + data[i * n_step + j * K + m][1] + data[i * n_step + j * K + m][2]
            if m != 0:
                if xyzsum[m] > xyzsum[mpi]:
                    mpi = m
                if xyzsum[m] < xyzsum[mvi]:
                    mvi = m
                aamvsum += abs(avm[m] - avm[m - 1])
            
            for n in range(xyz):
                saa[n] += data[i * n_step + j * K + m][n]
                sma += data[i * n_step + j * K + m][n]
                ta[m][n] = math.acos(data[i * n_step + j * K + m][n] / avm[m])
                tasum[n] += ta[m][n]
                tasquare[n] += pow(ta[m][n], 2)
            
        calculate[int(k / t) * i + j][0] = aamvsum / K
        calculate[int(k / t) * i + j][1] = 0
        calculate[int(k / t) * i + j][2] = mpi
        calculate[int(k / t) * i + j][3] = mvi
        calculate[int(k / t) * i + j][4] = j
        calculate[int(k / t) * i + j][5] = sma / K
        calculate[int(k / t) * i + j][6] = statistics.median(xyzsum)
        calculate[int(k / t) * i + j][7] = K
        calculate[int(k / t) * i + j][8] = saa[0]
        calculate[int(k / t) * i + j][9] = saa[1]
        calculate[int(k / t) * i + j][10] = saa[2]
        calculate[int(k / t) * i + j][11] = sma
        calculate[int(k / t) * i + j][12] = (tasquare[0] - tasum[0] * tasum[0] / K) / (K - 1)
        calculate[int(k / t) * i + j][13] = (tasquare[1] - tasum[1] * tasum[1] / K) / (K - 1)
        calculate[int(k / t) * i + j][14] = (tasquare[2] - tasum[2] * tasum[2] / K) / (K - 1)
                
data = pd.DataFrame(calculate)

data.to_csv("./dataset/training_data_LSTM_FS_2_wrist.csv", sep=",", header=False, index=False)

c:\users\a jin cho\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
